In [1]:
import numpy as np

import rte
import vegas_params as vp
import pylab as plt
from tqdm.auto import tqdm

In [2]:
#calculate the factor
R = 0.21
S_factor = np.pi * R**2
S_factor = 1

In [3]:
v_expected = {1e-6: 3.34332961e-07, 1e-8:1.10669159e+04}

#initial setup for source and target
src = rte.Source(R=vp.Vector([0,0,0]),
                 T=0,
                 s=vp.Vector([0,0,1]))

tgt = rte.Target(R=vp.Vector([1,0,1]),
                 T=1e-7,
                 s=vp.Vector([0,0,1]))

p = rte.Process(src, tgt, medium=rte.medium.water, Nsteps=2, use_uniform_sampling=True)
#p['steps']['t'] = rte.steps.Times_from_xi([1,0])
#p['steps']['s']['p_1'] = vp.Direction(cos_z, phi)
p

Process[3](
     --> src=Source[0](
         --> R=Vector[0](
             --> xyz=Fixed[0]([[0 0 0]])
        )
         --> T=Fixed[0]([[0]])
         --> s=Vector[0](
             --> xyz=Fixed[0]([[0 0 1]])
        )
    )
     --> tgt=Target[0](
         --> R=Vector[0](
             --> xyz=Fixed[0]([[1 0 1]])
        )
         --> T=Fixed[0]([[1.e-07]])
         --> s=Vector[0](
             --> xyz=Fixed[0]([[0 0 1]])
        )
    )
     --> steps=StepsUniform[3](
         --> t=Times_from_xi[1](
             --> xi=Concat[1](
                 --> p_0=Fixed[0]([[1]])
                 --> p_1=Uniform[1]([0, 1])
            )
        )
         --> s=Concat[2](
             --> p_0=Direction[0](
                 --> cos_theta=Fixed[0]([[1]])
                 --> phi=Fixed[0]([[0]])
            )
             --> p_1=Direction[2](
                 --> cos_theta=Uniform[1]([-1, 1])
                 --> phi=Uniform[1]([0.0, 6.283185307179586])
            )
        )
    )
)

# Cross-check with the fixed grid

In [4]:
def calc_value(p, xi, cos_z, phi, **kwargs):
    p['steps']['t'] = rte.steps.Times_from_xi([1,xi])
    p['steps']['s'] = vp.Direction(1, 0)|vp.Direction(cos_z, phi)
    return p.calculate(**kwargs)

In [5]:
xis =np.linspace(0,1,121)
coss=np.linspace(0,1,101)
phis=np.linspace(0,2*np.pi, 40)

result = [[[calc_value(p,xi,cos_z,phi).mean for phi in phis]
           for cos_z in coss]
          for xi in tqdm(xis)]
res0 = np.asarray(result)
res0.shape

In [6]:
p = rte.Process(src, tgt, medium=rte.medium.water, Nsteps=2, use_uniform_sampling=True)

def calc_values(p, xi, cos_z, phi, **kwargs):
    x = np.stack([x.flatten() for x in np.meshgrid(xi, cos_z, phi, indexing='ij')]).T
    shape = (len(xi),len(cos_z),len(phi))
    res =  p.__construct__(x).squeeze()*p.factor
    res = np.reshape(res,shape)
    return res

In [7]:
%%time
res1 = calc_values(p,xis,coss,phis)

2024-04-22 12:37:53.573 | DEBUG    | rte.steps:Times_from_xi:37 - powers = [[-1]
 [ 0]]
/home/ash/work/baikal/rte/RTE_package/rte/process.py:69: RuntimeWarning: invalid value encountered in divide
  step_length = np.where(is_linear, -R1_2/(2*R1N), (R1N+np.sqrt(D))/(gamma))
2024-04-22 12:37:53.997 | DEBUG    | rte.process:__call__:113 - scat_factor=[[0.00720776]
 [0.00719315]
 [0.00714998]
 ...
 [0.00221187]
 [0.00221187]
 [0.00221187]]
2024-04-22 12:37:53.997 | DEBUG    | rte.process:__call__:114 - att_factor=[[[0.02560935]
  [0.02560935]
  [0.02560935]
  ...
  [0.02560935]
  [0.02560935]
  [0.02560935]]]
2024-04-22 12:37:53.998 | DEBUG    | rte.process:__call__:115 - delta_factor=[[[0.00020665]
  [0.00020665]
  [0.00020665]
  ...
  [0.00020665]
  [0.00020665]
  [0.00020665]]]


CPU times: user 458 ms, sys: 37.1 ms, total: 495 ms
Wall time: 497 ms


### Направление последнего шага $\vec{s}'$

Теперь направление определяется так:
$$
\vec{s}^* = \frac{\vec{R_1}}{L}+\vec{N}
$$

# Check factors

In [8]:
p.save_trajectory=True
calc_value(p, xi=0.1, cos_z=1, phi=0)

2024-04-22 12:37:54.044 | DEBUG    | rte.steps:Times_from_xi:37 - powers = [[-1]
 [ 0]]
2024-04-22 12:37:54.046 | DEBUG    | rte.process:__call__:113 - scat_factor=[[0.00221187]]
2024-04-22 12:37:54.046 | DEBUG    | rte.process:__call__:114 - att_factor=[[[0.02560935]]]
2024-04-22 12:37:54.046 | DEBUG    | rte.process:__call__:115 - delta_factor=[[[0.00020665]]]


38.8779(0)

$$
F_{att} = e^{-\mu_t\,c\,T}\cdot \left[\mu_s\,c\,T\right]^n = 0.025609348656401597
$$

$$
F_{scat} = \prod_{i=1}^{n} f(\hat{s}_i\cdot\hat{s}_{i-1}) = 0.03344281543575628
$$

$$
F_{delta} = \frac{1}{L\cdot|(\vec{R}_1\vec{N} - L(1-\vec{N}^2)|} \cdot \frac{1}{cT} = 0.00020665
$$
where $L$ is last step:
$$
L\equiv |\vec{R}-\vec{r}_{n-1}| = |\vec{R}-\vec{r}_{2}| = 10.50698152
$$
and $N$ is direction vector, scaled by time:
$$
\vec{N} = \sum\limits_{i=0}^{n-2}\vec{s}_i\cdot \frac{t_{i+1}-t_{i}}{t_{n-1}} = [0, 0, 1]
$$

Result:
$$
F_{total} = F_{att}\cdot F_{scat}\cdot F_{delta}\cdot c = 38.87785328
$$

In [9]:
#grab the variables
from rte.medium import water
T = 1e-7
s = p.trajectory.s.swapaxes(0,1)
r = p.trajectory.R.swapaxes(0,1)
t = p.trajectory.T.swapaxes(0,1)
R = r[-1]


In [10]:
#att factor
F_att = rte.medium.water.attenuation(T,2)
F_att

0.025609348656401597

In [11]:
#scat factor
s_dot_s = np.sum(s[1:]*s[:-1], axis=2)
scat_factors = rte.medium.water.scatter(s_dot_s).flatten()
F_scat = np.prod(scat_factors)
F_scat

0.03344281543575628

In [ ]:
#delta factor
dR_last = np.diff(r,axis=0)[-1]
L = dR_last.mag()
dt = np.diff(t, axis=0)/t[2]
N = np.sum((s*dt)[:-1], axis=0)
R1 = R-N*T*water.c
R1N = R1.dot(N)
gamma = 1-np.linalg.norm(N,axis=-1)

F_delta = 1/(L*np.abs(R1N - L*gamma)) 
F_delta *= 1/(water.c*T)
F_delta.squeeze()

In [ ]:
L

In [ ]:
F_att*F_delta*F_scat*water.c

In [ ]:
import textwrap

def print_factors(p):
    res = f'{p.factor}'
    if hasattr(p,'parameters'):
        pars = '\n'.join([f' --> {name}={print_factors(par)}' for name,par in p.parameters.items()])
        res += f'(\n{textwrap.indent(pars,"    ")}\n)'
    return res


print(print_factors(p))

In [ ]:
p['steps'].factor
p['steps']['s'].factor

In [ ]:
for val in ['R','T','s']:
    print(f'{val} = \n{p.trajectory.__dict__[val]}')

In [ ]:
np.savez('check_grid_1e-7.npz',phi=phis, xi=xis, cos_theta=coss, result=res1, process_desc=str(p))

In [ ]:
f = np.load('check_grid.npz')
print(f['process_desc'])

In [ ]:
res0[:,0,0]

In [ ]:
res1[:,0,0]

In [ ]:
fname = 'cross_check/Light_1_0_1_0.9.npz'
fname = 'cross_check/Light_1_0_2_0.9.npz'
fname = 'cross_check/Light_2_0_2_0.9.npz'
fname = 'cross_check/SecondLight_1_0_1_0.9.npz'
#fname = 'cross_check/Light_10_0_10_0.9.npz'

#fname = 'cross_check/Light_3_0_10_0.9.npz'
#fname = 'cross_check/Light_5_0_2_0.9.npz'

In [ ]:
#load the file
data = np.load(fname)

In [ ]:
#initial setup for source and target
src = rte.Source(R=vp.Vector([0,0,0]),
                 T=0,
                 s=vp.Vector([0,0,1]))

tgt = rte.Target(R=vp.Vector(data['r']),
                 T=1e-7,
                 s=vp.Vector([0,0,1]))

In [ ]:
#take every n-th point
n = 1
#select some points
#times = data['time'][::n]
xis = data['ksi'][::n]
photons = data['photons'][:,:,::n]
#skip 0-th order, stop at 4th 
photons = photons[1:5]
#correct by factor 
photons = photons/S_factor
photons.shape

In [ ]:
xis.shape

In [ ]:
Nsteps_total = photons.shape[0]
Ns = np.arange(Nsteps_total)+1

# Cross-check with $\xi$

In [ ]:
from tqdm.auto import tqdm

result = []
Nsteps=2
p = rte.Process(src, tgt, medium=rte.medium.water, Nsteps=Nsteps)
for xi in tqdm(xis, desc=f'{Nsteps} order'):
    #p['tgt']['T']=vp.Fixed(t)
    p['steps']['t']['xi']['p_1']=vp.Fixed(xi)
    res = p.calculate(nitn=20, neval=100000)
    result.append([res.mean, res.sdev])

#make it the same shape as photons
result = np.array(result)
result = np.swapaxes(result, 0,1)
result.shape

In [ ]:
p

In [ ]:
import pylab as plt

fig = plt.figure(figsize=(6,4))#, sharey=True)

old = photons[1]
new = result
plt.errorbar(x=xis, y=old[0], yerr=old[1], fmt='.-k', label='V')
plt.errorbar(x=xis, y=new[0], yerr=new[1], fmt='.r-', label='A')
    
plt.ylabel('$\Phi^{'+f'({Nsteps})'+'}$')

#plt.sca(axes[0])
#plt.plot(c_vals.keys(), c_vals.values(), '*b', label='Data1')

plt.yscale('log')
plt.grid(ls=':')
plt.legend()

#plt.ylim(1e-2)
plt.xlabel('$xi$')
plt.suptitle(fname.rsplit('/',1)[-1])
plt.tight_layout()

plt.show()


# Cross-check with times

In [ ]:
from tqdm.auto import tqdm

result = []
for Nsteps in tqdm(Ns):
    result_n = []
    p = rte.Process(src, tgt, medium=rte.medium.water, Nsteps=Nsteps)
    for xi in tqdm(xis, desc=f'{Nsteps} order'):
        #p['tgt']['T']=vp.Fixed(t)
        p['steps']['t']['xi']['p_1']=
        res = p.calculate(nitn=10, neval=10000)
        result_n.append([res.mean, res.sdev])
    result.append(result_n)

#make it the same shape as photons
result = np.array(result)
result = np.swapaxes(result, 1,2)

In [ ]:
f = [1,1,2**(0.5),2,2**(2)]

In [ ]:
c_vals = {1e-6: 3.34332961e-07, 1e-8:1.10669159e+04}

In [ ]:
fig, axes = plt.subplots(2,len(Ns)//2, figsize=(8,6))#, sharey=True)
axes = axes.flatten()
ax = iter(axes)
import pylab as plt
for N,old,new in zip(Ns, photons, result):
    plt.sca(next(ax))
    plt.errorbar(x=times, y=old[0], yerr=old[1], fmt='.k', label='V')
    plt.errorbar(x=times, y=new[0], yerr=new[1], fmt='r-', label='A')
    if(f[N]!=1):
        plt.errorbar(x=times, y=new[0]/f[N], yerr=new[1]/f[N], fmt='r--', label=f'A/{f[N]}')
    plt.ylabel('$\Phi^{'+f'({N})'+'}$')

#plt.sca(axes[0])
#plt.plot(c_vals.keys(), c_vals.values(), '*b', label='Data1')
for ax in axes:
    plt.sca(ax)
    plt.yscale('log')
    plt.grid(ls=':')
    plt.legend()

plt.xlabel('Time, s')
plt.suptitle(fname.rsplit('/',1)[-1])
plt.tight_layout()

plt.show()



In [ ]:
fig, axes = plt.subplots(2,len(Ns)//2, figsize=(8,6))#, sharey=True)
axes = axes.flatten()
ax = iter(axes)

ratio = result/photons

for N, rat in zip(Ns, ratio):
    plt.sca(next(ax))
    plt.errorbar(x=times, y=rat[0], fmt='r-', label='new/old')
    #plt.yscale('log')
    plt.axhline(y=1,ls='-', color='k', lw=2)
    plt.grid(ls=':')
    plt.ylabel(f'new/old, N={N}')
    
#axes[0].legend()
plt.xlabel('Time, s')
plt.suptitle(fname.rsplit('/',1)[-1])
plt.tight_layout()

plt.show()



In [ ]:
#plot the ratios by order
r_valid =np.ma.masked_invalid(ratio[:,0])
r_mean = r_valid.mean(axis=-1)
r_sdev = r_valid.std(axis=-1)
plt.errorbar(x=Ns, y=r_mean, yerr=r_sdev, fmt='*k', label='ratio from data')
plt.plot(Ns, 2.0**(Ns-2), '--r', label = '$2^{N-2}$')
plt.plot(Ns, 2**((Ns-1)/2), '--b', label = '$2^{(N-1)/2}$')
plt.grid(ls=':')
plt.legend()
plt.xlabel('N')
plt.ylabel('new/old')
plt.axhline(y=1)
plt.show()